# ⚙️ PROCESSAMENTO DOCLING - Etapa 3/6

## 📋 O que este notebook faz

Este notebook **extrai texto limpo** dos documentos baixados usando a biblioteca Docling:

- 🔍 **Descobre documentos** suportados (.pdf, .md, .txt) em `pipeline-data/documents/`
- 📄 **Arquivos Markdown** → Copia diretamente (já estão em formato texto)
- 📕 **PDFs e outros** → Extrai texto usando Docling com OCR automático
- 🧹 **Unifica formato** → Tudo vira Markdown (.md) para processamento uniforme
- 📂 **Preserva estrutura** → Mantém hierarquia de pastas original

## 🔧 Como funciona

**Docling** é uma biblioteca avançada que:
- Detecta automaticamente quando usar OCR
- Preserva formatação e estrutura dos documentos  
- Suporta português e inglês
- Exporta para Markdown limpo e estruturado

## 📊 Output esperado

~31 arquivos processados em `pipeline-data/processed/` (todos em formato .md)

---

## 🔍 Descoberta e Preparação

In [1]:
from pathlib import Path
from docling.document_converter import DocumentConverter
import time
from datetime import datetime

# Marcar início da execução
stage_start = time.time()
start_timestamp = datetime.now().isoformat() + "Z"

# Extensões suportadas
SUPPORTED_EXTENSIONS = ['.pdf', '.md', '.txt']

# Diretórios
docs_dir = Path("pipeline-data/documents")
processed_dir = Path("pipeline-data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório processado recursivamente
for f in processed_dir.rglob("*"):
    if f.is_file():
        f.unlink()

# Listar documentos recursivamente (apenas com extensões suportadas)
all_files = list(docs_dir.rglob("*"))
documents = [f for f in all_files if f.is_file() and f.suffix.lower() in SUPPORTED_EXTENSIONS]

print(f"Documentos encontrados: {len(documents)}")
for doc in documents[:10]:
    # Mostrar o caminho relativo ao docs_dir
    relative_path = doc.relative_to(docs_dir)
    print(f"  {relative_path}")

if len(documents) > 10:
    print(f"  ... e mais {len(documents) - 10} documentos")

Documentos encontrados: 31
  30-Aprovados/Mapas/Visão Geral do Self Checkout.md
  30-Aprovados/Mapas/Visão Geral do NIC.md
  30-Aprovados/Mapas/Processa Sistemas.md
  30-Aprovados/Tópicos/Cancelamento de cupom.md
  30-Aprovados/Tópicos/Solicitação de ajuda.md
  30-Aprovados/Tópicos/Histórico de atualizações Self Checkout.md
  30-Aprovados/Tópicos/Funcionalidade do bloqueio.md
  30-Aprovados/Tópicos/Aplicação de desconto por item.md
  30-Aprovados/Tópicos/Propósito do NIC.md
  30-Aprovados/Tópicos/Reimpressão do último cupom.md
  ... e mais 21 documentos


## ⚙️ Processamento e Extração

In [2]:
import shutil

# Processar documentos
converter = DocumentConverter()
processed_count = 0
errors = []
copied_count = 0

for doc_path in documents:
    try:
        # Criar caminho mantendo a estrutura de pastas
        relative_path = doc_path.relative_to(docs_dir)
        output_path = processed_dir / relative_path.parent
        output_path.mkdir(parents=True, exist_ok=True)
        
        # Verificar se é arquivo Markdown
        if doc_path.suffix.lower() in ['.md', '.markdown']:
            # Copiar arquivo Markdown diretamente
            output_file = output_path / doc_path.name
            shutil.copy2(doc_path, output_file)
            
            copied_count += 1
            print(f"  ✅ Copiado: {doc_path.name} ")
        else:
            # Para outros formatos, usar Docling
            result = converter.convert(doc_path)
            
            # Extrair texto
            text_content = result.document.export_to_markdown()
            
            # Salvar texto extraído
            text_file = output_path / f"{doc_path.stem}.md"
            with open(text_file, "w", encoding="utf-8") as f:
                f.write(text_content)
            
            processed_count += 1
            print(f"  ✅ Convertido: {doc_path.name}")
        
    except Exception as e:
        error_msg = f"{doc_path.name}: {str(e)}"
        errors.append(error_msg)
        print(f"  ❌ {doc_path.name} falhou: {str(e)}")
    
print(f"\n📊 Resumo:")
print(f"  Convertidos com Docling: {processed_count}")
print(f"  Markdown copiados: {copied_count}")
print(f"  Total processados: {processed_count + copied_count}")
print(f"  Erros: {len(errors)}")

if errors:
    print(f"\n❌ Erros:")
    for error in errors:
        print(f"  {error}")

  ✅ Copiado: Visão Geral do Self Checkout.md 
  ✅ Copiado: Visão Geral do NIC.md 
  ✅ Copiado: Processa Sistemas.md 
  ✅ Copiado: Cancelamento de cupom.md 
  ✅ Copiado: Solicitação de ajuda.md 
  ✅ Copiado: Histórico de atualizações Self Checkout.md 
  ✅ Copiado: Funcionalidade do bloqueio.md 
  ✅ Copiado: Aplicação de desconto por item.md 
  ✅ Copiado: Propósito do NIC.md 
  ✅ Copiado: Reimpressão do último cupom.md 
  ✅ Copiado: Função do Chat NIC.md 
  ✅ Copiado: Acesso ao menu do fiscal.md 
  ✅ Copiado: Cronograma e marcos do projeto.md 
  ✅ Copiado: Efetuar pagamento.md 
  ✅ Copiado: Cancelamento de item.md 
  ✅ Copiado: Operação contínua e evolução do sistema.md 
  ✅ Copiado: Finalidade e visão do NIC.md 
  ✅ Copiado: Iniciar a compra.md 
  ✅ Copiado: Processo de documentação e validação.md 
  ✅ Copiado: Infraestrutura técnica e operacional.md 
  ✅ Copiado: Apresentação do sistema Self Checkout.md 
  ✅ Copiado: Retornar para registro de venda.md 
  ✅ Copiado: Componentes principa

## 📊 Relatório de Execução

In [3]:
import json

# Calcular duração
stage_duration = time.time() - stage_start

# Carregar relatório existente
report_path = Path("pipeline-data/report.json")
if report_path.exists():
    with open(report_path, "r") as f:
        report = json.load(f)
else:
    report = {"stages": [], "context": {}, "summary": {}}

# Adicionar informações desta etapa
stage_report = {
    "stage": 3,
    "name": "Processamento Docling",
    "status": "SUCCESS" if len(errors) == 0 else "FAILED",
    "start_time": start_timestamp,
    "duration_seconds": round(stage_duration, 2),
    "results": {
        "documents_found": len(documents),
        "markdown_copied": copied_count,
        "docling_converted": processed_count,
        "total_processed": processed_count + copied_count,
        "processing_errors": len(errors)
    }
}

# Se houve erros, adicionar detalhes
if errors:
    stage_report["errors"] = errors[:5]  # Limitar a 5 erros

# Adicionar ou atualizar stage no relatório
stages_updated = False
for i, stage in enumerate(report["stages"]):
    if stage["stage"] == 3:
        report["stages"][i] = stage_report
        stages_updated = True
        break

if not stages_updated:
    report["stages"].append(stage_report)

# Atualizar timestamp
report["summary"]["last_update"] = datetime.now().isoformat() + "Z"

# Salvar relatório atualizado
with open(report_path, "w") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"📊 Relatório atualizado: {report_path}")
print(f"⏱️ Duração da etapa: {stage_duration:.2f}s")

📊 Relatório atualizado: pipeline-data/report.json
⏱️ Duração da etapa: 0.06s
